<a href="https://colab.research.google.com/github/jfr11101/MyProjects/blob/main/Weather_Data_Python_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import requests
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='Jafar Mirahmadi')
api_key = '9ebe41a55a60d357c3e2905adbf5fc82'

cities = ['Tehran','Buenos Aires','Copenhagen', 'London', 'Paris', 'Berlin', 'Athens', 'Mahallat', 'Dublin', 'Tokyo',
          'Luxembourg', 'Mexico City', 'Oslo', 'Lisbon', 'Bucharest', 'San Marino', 'Belgrade', 'Seoul', 'Madrid', 'Bern',
          'Stockholm', 'Bangkok', 'Taipei', 'Ankara', 'Abu Dhabi', 'Washington D.C.', 'Montevideo', 'Tashkent', 'Hanoi', 'Cardiff',
          'Vienna', 'Yerevan', 'Nassau', 'Rome', 'Kingston', 'Amman', 'Astana', 'Nairobi', 'Pristina', 'Bishkek', 'Vientiane', 'Riga',
          'Maseru', 'Monrovia', 'Tripoli', 'Antananarivo', 'Male', 'Majuro', 'Podgorica', 'Amsterdam']


In [ ]:
def get_lat_lon(city):
    location = geolocator.geocode(city)
    lat = location.raw['lat']
    lon = location.raw['lon']
    return (lat, lon)

In [ ]:
def get_weather(cities, api_key):

    coordinates = [get_lat_lon(city) for city in cities]
    weather_data = []

    filename = f"weather.csv"
    with open(filename, "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Coordinates", "City Name", "Country",
                         "Temperature (°С)", "Minimum Temperature",
                         "Maximum Temperature", "Humidity (%)",
                         "Wind Speed (m/s)", "Description", "Next 5 Days"]) # Write header

        for i in range(len(coordinates)):
            lat = coordinates[i][0]
            lon = coordinates[i][1]
            base_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"

            response = requests.get(base_url)

            data = response.json()

            cityName        = data['name']
            country         = data['sys']['country']
            temp            = data['main']['temp'] - 273.15 # change to °С
            temp_min        = data['main']['temp_min'] - 273.15
            temp_max        = data['main']['temp_max'] - 273.15
            wind_speed      = data['wind']['speed']
            humidity        = data['main']['humidity']
            current_weather = data['weather'][0]['description']
            coordinate      = data['coord']

            next5Days = [] #5 Day Forecast List
            base_url = f'http://api.openweathermap.org/data/2.5/find?q={cityName}&appid={api_key}' # Find ID of city to search in forecast
            response = requests.get(base_url)
            data = response.json()
            id = data['list'][0]['id'] # get id of city

            base_url = f"http://api.openweathermap.org/data/2.5/forecast?id={id}&appid={api_key}"
            response = requests.get(base_url)
            data = response.json()

            for dt in range(40):
                  dt_tuple = (data['list'][dt]['dt_txt'],
                              data['list'][dt]['main']['temp_min']-273.15,
                              data['list'][dt]['main']['temp_max']-273.15,
                              data['list'][dt]['main']['humidity'],
                              data['list'][dt]['weather'][0]['description'],
                              data['list'][dt]['wind']['speed'])
                  next5Days.append(dt_tuple)

            writer.writerow([coordinate, cityName, country,
                             temp, temp_min, temp_max,
                             humidity, wind_speed,
                             current_weather, next5Days])



In [ ]:
get_weather(cities, api_key)